<a href="https://colab.research.google.com/github/madhulsachdeva/Meeting_Minutes_From_Audio_Recording/blob/main/Meeting_Minutes_from_Audio_Recording.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2 markdown gradio

In [ ]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
import gradio as gr

In [ ]:
# Define 2 LLM Models.
AUDIO_MODEL = "whisper-1"

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
QWEN2="Qwen/Qwen2-7B-Instruct"

system_prompt="You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways, action items with owners and next steps, respond in markdown."
user_prompt = "Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; action items with owners."

g_min =""
g_min_clean=""

In [ ]:
# Sign in to HuggingFace Hub
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

# Sign in to OpenAI using Secrets in Colab
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

# Mount Google Drive
drive.mount("/content/drive")

In [ ]:
file="denver_extract.mp3"
temp_file_path=f"/content/drive/{file}"
print(temp_file_path)


In [ ]:
os.path.exists("/content/drive/MyDrive/denver_extract.mp3")
 #("f/content/drive/{temp_file_path}")

In [ ]:
#Define System Prompt

def get_system_message(s_message):
  system_messages = [
      {"role": "system", "content": s_message}
    ]
  return system_messages

# Define User Prompt
def get_user_message(u_message, transcribed_script):
  user_prompt = f"{u_message}\n{transcribed_script}"
  user_messages = [
      {"role": "user", "content": user_prompt}
    ]
  return user_messages

#Get Chat Message Template
def get_messages(sprompt,uprompt,transcription):
  messages=get_system_message(sprompt)+get_user_message(uprompt,transcription)
  return messages

#Quantize model
def get_quant_config():
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
  return quant_config

# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3
def process_audiofile(filename):
  audio_filename = f"/content/drive/MyDrive/{filename}"
  return audio_filename
  #denver_extract.mp3"

def transcribe_audio(audio_filename):
  audio_file = open(audio_filename, "rb")
  transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
  return transcription

def generate(model, messages):
    tokenizer = AutoTokenizer.from_pretrained(model)
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    model = AutoModelForCausalLM.from_pretrained(model, device_map="auto",quantization_config=get_quant_config())
    # Define streamer within the second generate function as well
    #streamer = TextStreamer(tokenizer)
    outputs = model.generate(inputs, max_new_tokens=3000)#,streamer=streamer)
    del inputs, model
    torch.cuda.empty_cache()

    return tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

def remove_headers(text, word):
    # Split the text at the specified word
    parts = text.split(word)
    # If the word is found, return the text from the word onwards
    if len(parts) > 1:
        return word + parts[1]
    else:
        return text  # Return original text if the word is not found

def clean_up_text(text_to_clean):
    meeting_index = text_to_clean.find("**M")
    if meeting_index == -1:
        return text_to_clean
    else:
        return text_to_clean[meeting_index:]

In [ ]:
"""
def generate(model, messages):
    tokenizer = AutoTokenizer.from_pretrained(model)
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    model = AutoModelForCausalLM.from_pretrained(model, device_map="auto",quantization_config=get_quant_config())
    # Define streamer within the second generate function as well
    streamer = TextStreamer(tokenizer)
    outputs = model.generate(inputs, max_new_tokens=3000, streamer=streamer)
    del inputs, model
    torch.cuda.empty_cache()
    return tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
"""

In [ ]:
def generate_minutes(model,filename):
  file = filename
  audio_filepath = ''
  sprompt =''
  uprompt =''
  quant_config = get_quant_config()
  temp_file_path=f"/content/drive/MyDrive/{file}"
  print(temp_file_path)

  if os.path.exists(temp_file_path):
    audio_filepath = process_audiofile(file)
    transcription = transcribe_audio(audio_filepath)
    sprompt = get_system_message(system_prompt)
    uprompt = get_user_message(user_prompt,transcription)
    messages= sprompt+uprompt

    minutes = generate(model,messages)

    #Remove unwanted text.
    g_min_clean = clean_up_text(minutes)
    return g_min_clean

  else:
    return f"File not found, please make sure \'{file}\' exists in your Google Drive"


In [ ]:
def stream_minutes(model, filename):
    """
    Stream minutes generation with Gradio compatibility

    Args:
        model: The LLM model to use for generation
        filename: Input file to process

    Yields:
        str: Streamed text chunks for Gradio interface
    """
    stream = generate_minutes(model, filename)
    response_text = ""

    for chunk in stream:
        response_text += chunk
        # Format as markdown if needed
        markdown_text = f"# Meeting Minutes\n\n{response_text}"
        yield markdown_text

In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft(),title="📑Easy Minutes") as demo:
    gr.Markdown("# 📆Meeting Minutes Generator \nEasily generate professional looking meeting minutes from audio recordings of your meetings.")

    with gr.Row():
        # Inputs
        with gr.Column():
            model = gr.Textbox(
                value=LLAMA,
                visible=False,
                label="Model",
                container=True
            )
            filename = gr.Textbox(
                label="Meeting recording filename",
                placeholder="enter name of audio meeting recording file located in GDrive",
                lines=1,
                container=True
            )
            with gr.Row():
                process_button = gr.Button("Generate Minutes", variant="primary")
                clear_button = gr.Button("Clear", variant="secondary")

        # Outputs
        with gr.Column():
            output = gr.Markdown(
                value="## **Minutes will appear below once processed...** \n❕Please note this can take 5 - 7 minutes depending on the size of the file",  # Markdown formatting for bold heading
                label="Meeting Minutes",
                container=True
            )

    # Clear function
    def clear_outputs():
        return {
            filename: "",
            output: f"## **Minutes will appear below once processed...** \n❕Please note this can take 5 - 7 minutes depending on the size of the file"
        }

    # Connect the components
    process_button.click(
        fn=stream_minutes,
       inputs=[model, filename],
        outputs=output,
        show_progress=True
    )

    # Connect clear button
    clear_button.click(
        fn=clear_outputs,
        inputs=[],
        outputs=[filename, output]
    )

demo.queue()
demo.launch(inbrowser=True, debug=True, share=True)